# Contacts2: meervoudige waarden

In een MongoDB-document kan een property meerdere waarden bevatten, in de vorm van een array van waarden.

> In een relationele (SQL) database heb je voor meervoudige waarden een aparte tabel nodig. In MongoDB kun je zo'n array direct in het document opnemen (*embedding*).

* hoe geef je deze meervoudige waarden weer in een document?
* hoe geef je deze weer in een validator-schema?
* welke gevolgen heeft dit voor queries?
    * voor het bevragen van de data
    * voor het toevoegen van data
    
**TODO** dit onderdeel verder afmaken: is nu incompleet.    

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

from pymongo import MongoClient
print('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

mongopathfile = !cat mongopath
mongopath = mongopathfile[0]

collection.drop()
os.system(mongopath + 'mongoimport -d ' + dbname + ' -c contacts adressen.json')

In [ ]:
mirjam = {"name": "Mirjam Arendsen",
          "tel": [{"tel": "06-1234 5678", "soort": "mobiel"},
                   {"tel": "020-8877669", "soort": "thuis"}],
          "email": "mirjaar@example.com",
          "address": {"street": "Damesgracht 42", "city": "Rommeldam"}}

In [ ]:
collection.insert_one(mirjam)

In [ ]:
list(collection.find())

In [ ]:
list(collection.find({"tel": {"$exists": True}}))

In [ ]:
list(collection.find({"tel.tel": "06-1234 5678"}))

In [ ]:
list(collection.find({"$or": [{"tel.tel": "010-123 123 9"}, {"tel": "010-123 123 9"}]}))

**Opmerking**: ik zie geen andere oplossing dan deze wat omslachtige query. Een alternatief is om het schema te veranderen, zodat we altijd een array gebruiken (dat dan overigens ook leeg kan zijn).

Welke conventie kunnen we hierbij hanteren? gebruiken we een speciale naam voor het array? Ik voel er veel voor om de "gestandaardiseerde" naam voor het eigenlijke veld te gebruiken: bijvoorbeeld: "tels:telephone". Dit komt ook overeen met het gebruik in RDFa en microdata: je vermeldt de eigenschap dan meerdere keren. (In het geval van JSON moet je dan een naam voor het array verzinnen.)
(In schema.org zijn de meervouden zoveel mogelijk vervangen door het enkelvoud, bijvoorbeeld: parent in plaats van parents. Je hebt dan met een relatie te maken die mogelijk meervoudig kan zijn.)